In [1]:
#Download the file from github
!git clone https://github.com/god441347/email-bot-comprehend-class.git
!mkdir email-bot-comprehend-class/data
!mv ./email-bot-comprehend-class/Comprehend_Training_Data.csv ./email-bot-comprehend-class/data/


from sagemaker import get_execution_role


Cloning into 'email-bot-comprehend-class'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
Unpacking objects: 100% (12/12), done.
remote: Total 12 (delta 5), reused 0 (delta 0), pack-reused 0


In [2]:
role = get_execution_role()
role
# S3 prefix
prefix = 'email-classification'
role

'arn:aws:iam::709753484661:role/service-role/AmazonSageMaker-ExecutionRole-20210513T160706'

In [3]:

import sagemaker as sage
sess = sage.Session()
WORK_DIRECTORY = 'email-bot-comprehend-class/data'
#loading the data to s3 bucket
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)
data_location

's3://sagemaker-us-west-2-709753484661/email-classification'

In [25]:
#Create email classification training job
import boto3

client = boto3.client('comprehend')

response = client.create_document_classifier(
    DocumentClassifierName='email-classifications-1',
    DataAccessRoleArn=role,
    InputDataConfig={
        'S3Uri': data_location
    },
    #ClientRequestToken='string',
    LanguageCode='en'
)

In [34]:
import time
print (response["DocumentClassifierArn"])
print (response)
modelarn=response["DocumentClassifierArn"]
response_des = client.describe_document_classifier(
    DocumentClassifierArn=modelarn
)
print (response_des['DocumentClassifierProperties']['Status'])
train_status=response_des['DocumentClassifierProperties']['Status']
print ("Training started")
while train_status!='TRAINED':
    print (".")
    time.sleep(30)
    
print("Training Completed")

arn:aws:comprehend:us-west-2:709753484661:document-classifier/email-classifications-1
{'DocumentClassifierArn': 'arn:aws:comprehend:us-west-2:709753484661:document-classifier/email-classifications-1', 'ResponseMetadata': {'RequestId': '984db751-58fa-4be1-bdd4-9e25d33d32e5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '984db751-58fa-4be1-bdd4-9e25d33d32e5', 'content-type': 'application/x-amz-json-1.1', 'content-length': '113', 'date': 'Fri, 14 May 2021 00:04:57 GMT'}, 'RetryAttempts': 0}}
TRAINING
Training started
.
.


KeyboardInterrupt: 

In [24]:
#create endpoint for the trained classifer
response_ep = client.create_endpoint(
    EndpointName='email-classifications',
    ModelArn=modelarn,
    DesiredInferenceUnits=1,
    #ClientRequestToken='string',
    Tags=[
        {
            'Key': 'Name',
            'Value': 'email classification'
        },
    ],
    DataAccessRoleArn=role
)

In [36]:
response_ep
epurn=response_ep["EndpointArn"]
epurn

'arn:aws:comprehend:us-west-2:709753484661:document-classifier-endpoint/email-classifications'

In [38]:
response_cd = client.classify_document(
    Text='Can you send the status of the transaction id:278960001',
    EndpointArn=epurn
)
response_cd

{'Classes': [{'Name': 'MONEYTRANSFER', 'Score': 0.6437000036239624},
  {'Name': 'PROMOCODE', 'Score': 0.3562999963760376}],
 'ResponseMetadata': {'RequestId': '0db2cb00-bb23-4b29-823d-c58823f64ac9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0db2cb00-bb23-4b29-823d-c58823f64ac9',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '113',
   'date': 'Fri, 14 May 2021 00:21:45 GMT'},
  'RetryAttempts': 0}}